In [9]:
import numpy as np
from cities.utils.cleaning_utils import find_repo_root
import pandas as pd
import numpy as np

root = find_repo_root()

In [10]:
import requests

# https://apps.bea.gov/api/signup/ link for registering in order to obtain BEA api key

api_key = "ED8C2AB7-DD09-4EE0-AC91-116A9E05348A"  # private api key for BEA data


years = "1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021"


# 100 stands for population

url = f"https://apps.bea.gov/api/data/?UserID={api_key}&method=GetData&datasetname=Regional&TableName=CAINC30&LineCode=100&Year={years}&GeoFips=MSA&ResultFormat=json"
response = requests.get(url)

assert response.status_code == 200  # 200 means success


data = response.json()
data_series = data["BEAAPI"]["Results"]["Data"]

df = pd.DataFrame(data_series)

In [14]:
# wrangle the data

population_ma = df.copy()

population_ma = population_ma[population_ma["GeoFips"].str.endswith("0")]

population_ma = population_ma[["GeoFips", "GeoName", "TimePeriod", "DataValue"]]
population_ma = population_ma.rename(
    columns={"DataValue": "Value", "TimePeriod": "Year", "GeoFips": "GeoFIPS"}
)
population_ma = population_ma.sort_values(by=["GeoFIPS", "Year"]).reset_index(drop=True)
population_ma["Value"] = population_ma["Value"].astype(float)
population_ma["GeoName"] = population_ma["GeoName"].astype(str)
population_ma["GeoFIPS"] = population_ma["GeoFIPS"].astype(np.int64)
population_ma["GeoName"] = (
    population_ma["GeoName"].str.extract(r"^(.*?)\s*\(", expand=False).fillna("")
)
population_ma["GeoName"] = population_ma["GeoName"] + " (MA)"

population_ma.head()

,GeoFIPS,GeoName,Year,Value
0,10180,"Abilene, TX (MA)",1993,152909.0
1,10180,"Abilene, TX (MA)",1994,153779.0
2,10180,"Abilene, TX (MA)",1995,156097.0
3,10180,"Abilene, TX (MA)",1996,156351.0
4,10180,"Abilene, TX (MA)",1997,157405.0


In [13]:
# transforming the data to wide format

population_wide = population_ma.pivot(
    index=["GeoFIPS", "GeoName"], columns="Year", values="Value"
)
population_wide = population_wide.reset_index()
population_wide.columns.name = None
population_wide.head()

,GeoFIPS,GeoName,1993,1994,1995,1996,1997,1998,1999,2000,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,10180,"Abilene, TX (MA)",152909.0,153779.0,156097.0,156351.0,157405.0,158264.0,159755.0,160288.0,...,168246.0,168614.0,169859.0,171579.0,172242.0,172915.0,174005.0,175187.0,176866.0,177829.0
1,10420,"Akron, OH (MA)",674114.0,678063.0,682146.0,687264.0,689461.0,691039.0,693125.0,695946.0,...,702245.0,703809.0,705134.0,704664.0,703862.0,704229.0,703925.0,703361.0,701625.0,696225.0
2,10500,"Albany, GA (MA)",148122.0,149356.0,150100.0,151596.0,152692.0,153449.0,153555.0,153585.0,...,154669.0,153680.0,153263.0,152018.0,151044.0,150387.0,150440.0,149530.0,148244.0,144922.0
3,10540,"Albany-Lebanon, OR (MA)",95496.0,96919.0,98853.0,100582.0,102054.0,102770.0,103462.0,103020.0,...,117865.0,117822.0,118285.0,119295.0,121648.0,123657.0,125788.0,127700.0,128978.0,129948.0
4,10580,"Albany-Schenectady-Troy, NY (MA)",827812.0,830817.0,830439.0,828007.0,824711.0,823712.0,824119.0,827399.0,...,879397.0,883854.0,886981.0,889918.0,892762.0,897172.0,899605.0,898791.0,899748.0,905369.0


In [15]:
# let's save obtained data in csv format

population_wide.to_csv(f"{root}/data/raw/population_ma.csv", index=False)